In [1]:
import spotipy
import pandas as pd
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
import config

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [4]:
results= sp.search(q="pop", limit =50, market ="DE")

In [5]:
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=pop&type=track&market=DE&offset=0&limit=50',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'},
       'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ',
       'id': '1Xyo4u8uXC1ZmMpatF05PJ',
       'name': 'The Weeknd',
       'type': 'artist',
       'uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/6tbjWDEIzxoDsBA1FuhfPW'},
       'href': 'https://api.spotify.com/v1/artists/6tbjWDEIzxoDsBA1FuhfPW',
       'id': '6tbjWDEIzxoDsBA1FuhfPW',
       'name': 'Madonna',
       'type': 'artist',
       'uri': 'spotify:artist:6tbjWDEIzxoDsBA1FuhfPW'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/5O0zUvdnJr0RbWzLFneN2i'},
     'href': 'https://api.spotify.com/v1/albums/5O0zUvdnJr0RbWzLFneN2i',
     'id': '5O0zUvdnJr0RbWzLFneN2

In [6]:
import pprint

In [7]:
pprint.pprint(results)

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=pop&type=track&market=DE&offset=0&limit=50',
            'items': [{'album': {'album_type': 'single',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'},
                                              'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ',
                                              'id': '1Xyo4u8uXC1ZmMpatF05PJ',
                                              'name': 'The Weeknd',
                                              'type': 'artist',
                                              'uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'},
                                             {'external_urls': {'spotify': 'https://open.spotify.com/artist/6tbjWDEIzxoDsBA1FuhfPW'},
                                              'href': 'https://api.spotify.com/v1/artists/6tbjWDEIzxoDsBA1FuhfPW',
                                  

In [8]:
list(results.keys())

['tracks']

In [9]:
list_of_songs=[]
for i in results["tracks"]["items"]:
    list_of_songs.append(i["id"])

song_features = sp.audio_features(list_of_songs)

df=pd.DataFrame(song_features)    
df=df[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

df.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms
0,0.854,0.674,-6.230,0.1840,0.05170,0.000085,0.5090,0.848,99.022,6WzRpISELf3YglGAh7TXcG,215467
1,0.548,0.805,-5.732,0.3510,0.21200,0.000405,0.4080,0.649,142.096,79s5XnCN4TJKTVMSmOx8Ep,216387
2,0.461,0.897,-3.982,0.0920,0.00821,0.000000,0.0965,0.358,165.107,4GssB27iJeqmfGxS94Tfij,220537
3,0.784,0.627,-14.617,0.0572,0.04250,0.717000,0.0645,0.952,136.879,45aoC0yE3RZGWz5QPb41M1,151440
4,0.769,0.818,-1.385,0.1920,0.26300,0.000000,0.0787,0.918,88.017,4E5IFAXCob6QqZaJMTw5YN,240800


In [10]:
query = "pop"
market = "DE"
limit = 50
offset = 0
results_per_offset = []
for i in range(5):  # Change range to control the number of iterations
    results = sp.search(q=query, limit=limit, market=market, offset=offset)
    results_per_offset.append(results)
    offset += 51

In [11]:
queries = ["pop", "pop hits"]  # List of different queries to switch between
market = "DE"
limit = 50
offset = 0
df_final = pd.DataFrame()  # This will hold the combined DataFrame from all offsets
rows_needed = 1000  # Number of rows required
current_row_count = 0  # To track the number of rows retrieved so far
max_offset = 950  # Set a limit for offset to avoid exceeding API's max offset of 1000
query_index = 0  # Index to switch between queries

# Loop to keep fetching results until we have 1000 rows
while current_row_count < rows_needed:
    try:
        # Switch queries if needed after reaching max offset for one query
        if offset > max_offset:
            query_index += 1
            if query_index >= len(queries):
                break  # Stop if we've exhausted all queries
            offset = 0  # Reset offset for the new query
        
        # Fetch search results for the current offset and query
        results = sp.search(q=queries[query_index], limit=limit, market=market, offset=offset)
        
        # Create an empty list to store the song IDs
        list_of_songs = []
        
        # Collect the song IDs from the search results
        for track in results["tracks"]["items"]:
            list_of_songs.append(track["id"])
        
        # Fetch audio features for the collected song IDs
        song_features = sp.audio_features(list_of_songs)
        
        # Create a DataFrame from the audio features and select relevant columns
        df = pd.DataFrame(song_features)
        df = df[["danceability", "energy", "loudness", "speechiness", "acousticness",
                 "instrumentalness", "liveness", "valence", "tempo", "id", "duration_ms"]]
        
        # Append the DataFrame to the final DataFrame
        df_final = pd.concat([df_final, df], ignore_index=True)
        
        # Update the current row count
        current_row_count = len(df_final)
        
        # Increment the offset by 51 for the next batch of results
        offset += 51
    
    except spotipy.exceptions.SpotifyException as e:
        print(f"SpotifyException occurred: {e}")
        break

# Slice the DataFrame to exactly 1000 rows if more than 1000 were retrieved
df_final = df_final.head(1000)

# Show the first few rows of the combined DataFrame
df_final.head()

# Optional: Save the DataFrame to a CSV file
df_final.to_csv('../01_data/df_final.csv', index=False)

df_final.to_csv("spotify_pop_songs_1000_rows.csv", index=False)

In [12]:
df_final.shape

(1000, 11)

In [13]:
results= sp.search(q="pop", limit =50, market ="DE", offset= 51)

In [14]:
list_of_songs=[]
for i in results["tracks"]["items"]:
    list_of_songs.append(i["id"])

song_features = sp.audio_features(list_of_songs)

df1=pd.DataFrame(song_features)    
df1=df1[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","id","duration_ms"]]
df1=df
df.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms
0,0.594,0.809,-6.217,0.0358,0.000107,0.249,0.0894,0.781,134.996,0M90VqUDEQ8TbbAc9TxHWg,234787
1,0.901,0.654,-10.150,0.0655,0.200000,0.000,0.1650,0.781,110.131,2ZqT3GcisHUBsEaqhLGHwU,180240
2,0.788,0.859,-2.724,0.0856,0.281000,0.000,0.0424,0.822,141.020,0KfKEiuaaMhv3nKn5zTUeR,207853
3,0.521,0.319,-10.329,0.0524,0.105000,0.000,0.0726,0.276,152.137,14wAM6xzQ7cWavgl1A7Qs1,144573
4,0.682,0.792,-6.689,0.1000,0.002600,0.000,0.0944,0.543,127.933,5ZGekegWxH1c6l5YRdcuTU,239120


In [15]:
results= sp.search(q="pop", limit =50, market ="DE", offset= 102)

In [16]:
list_of_songs=[]
for i in results["tracks"]["items"]:
    list_of_songs.append(i["id"])

song_features = sp.audio_features(list_of_songs)

df=pd.DataFrame(song_features)    
df=df[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","id","duration_ms"]]
df1=df
df.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms
0,0.646,0.876,-6.444,0.1970,0.00267,0.000000,0.171,0.2930,122.958,0NFBQhVLlzeY8ry9jUgHy2,185000
1,0.822,0.969,-11.471,0.0582,0.25900,0.000153,0.390,0.7650,128.007,4GewRLEiI4BDTQsoH8BRVV,283748
2,0.406,0.312,-13.478,0.0697,0.31000,0.003820,0.111,0.0595,143.831,77lVSrDVVjO7jt2P09UItx,199216
3,0.546,0.897,-3.067,0.1620,0.02600,0.000000,0.140,0.6420,173.187,1jVXXu8W4iny6MRkxKxxPI,191415
4,0.754,0.516,-8.619,0.2060,0.46200,0.000004,0.141,0.6070,103.920,5hl7ya200utFGWtsY8HApb,168812


In [17]:
df.shape

(50, 11)

In [18]:
from IPython.display import IFrame

track_id = '1Xyo4u8uXC1ZmMpatF05PJ'
#track_id= 'spotify:track:3hgl7EQwTutSm6PESsB7gZ'
IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [19]:
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [20]:
results = sp.search(q="The Weeknd")
play_song(results["tracks"]["items"][0]["id"])

In [21]:
playlist = sp.user_playlist_tracks("spotify", "4ebKOuGNfJ5g8RdtbEBHxe",market="GB")